## Module

In [1]:
import optuna
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

## Dataset

In [2]:
train = pd.read_csv('../input/titanic/train.csv')
print(train.shape)
train.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test = pd.read_csv('../input/titanic/test.csv')
print(test.shape)
test.head()

(418, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
submission = pd.read_csv('../input/titanic/gender_submission.csv')
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


## Info

In [5]:
train['Survived'].value_counts()

0    549
1    342
Name: Survived, dtype: int64

## Preprocess

In [6]:
features = [
    'Pclass',
    'Sex',
    'Age',
    'SibSp',
    'Parch',
    'Fare',
    'Embarked'
]

target = 'Survived'

In [7]:
for c in ['Sex', 'Embarked']:
    le = LabelEncoder()
    le.fit(train[c].fillna('nan'))
    
    train[c] = le.transform(train[c].fillna('nan'))
    test[c] = le.transform(test[c].fillna('nan'))

In [8]:
X_train = train[features]
y_train = train[target]
print(X_train.shape)
print(y_train.shape)

(891, 7)
(891,)


In [9]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=123)

print(X_train.shape)
print(y_valid.shape)
print(X_train.shape)
print(y_valid.shape)

(712, 7)
(179,)
(712, 7)
(179,)


## Params

In [10]:
def objective(trial):
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'eta': trial.suggest_uniform('eta', 0.05, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_uniform('min_child_weight', 0.1, 10),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 0.95),
        'subsample': trial.suggest_uniform('subsample', 0.6, 0.95)
    }
    
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_valid, label=y_valid)
    
    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    num_round = 500
    
    model = xgb.train(
        params,
        dtrain,
        num_round,
        evals=watchlist,
        early_stopping_rounds=20
    )

    y_pred_valid = model.predict(dvalid, ntree_limit=model.best_ntree_limit)
    
    score = log_loss(y_valid, y_pred_valid)
    return score

In [11]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=40)

[I 2023-10-24 08:27:24,811] A new study created in memory with name: no-name-fcfbc306-9e86-40eb-af26-f8794ce82241


[0]	train-logloss:0.66049	eval-logloss:0.66089
[1]	train-logloss:0.63184	eval-logloss:0.63210
[2]	train-logloss:0.60652	eval-logloss:0.60691
[3]	train-logloss:0.59480	eval-logloss:0.59618
[4]	train-logloss:0.57555	eval-logloss:0.57889
[5]	train-logloss:0.55726	eval-logloss:0.56088
[6]	train-logloss:0.54669	eval-logloss:0.55273
[7]	train-logloss:0.53234	eval-logloss:0.53832
[8]	train-logloss:0.51817	eval-logloss:0.52407
[9]	train-logloss:0.50631	eval-logloss:0.51258
[10]	train-logloss:0.49640	eval-logloss:0.50386
[11]	train-logloss:0.48717	eval-logloss:0.49664
[12]	train-logloss:0.47767	eval-logloss:0.48839
[13]	train-logloss:0.46916	eval-logloss:0.48125
[14]	train-logloss:0.46193	eval-logloss:0.47446
[15]	train-logloss:0.45518	eval-logloss:0.46868
[16]	train-logloss:0.44903	eval-logloss:0.46366
[17]	train-logloss:0.44382	eval-logloss:0.45927
[18]	train-logloss:0.43851	eval-logloss:0.45607
[19]	train-logloss:0.43465	eval-logloss:0.45218
[20]	train-logloss:0.42975	eval-logloss:0.44876
[2

[I 2023-10-24 08:27:25,082] Trial 0 finished with value: 0.41997993271277606 and parameters: {'eta': 0.07744067519636624, 'max_depth': 8, 'min_child_weight': 8.458230910952071, 'colsample_bytree': 0.9002809661679648, 'subsample': 0.8965381085744438}. Best is trial 0 with value: 0.41997993271277606.


[0]	train-logloss:0.65644	eval-logloss:0.65956
[1]	train-logloss:0.62389	eval-logloss:0.62675
[2]	train-logloss:0.59525	eval-logloss:0.59922
[3]	train-logloss:0.58072	eval-logloss:0.58858
[4]	train-logloss:0.55923	eval-logloss:0.56910
[5]	train-logloss:0.53849	eval-logloss:0.54993
[6]	train-logloss:0.52670	eval-logloss:0.54271
[7]	train-logloss:0.51027	eval-logloss:0.52823
[8]	train-logloss:0.49451	eval-logloss:0.51441
[9]	train-logloss:0.48127	eval-logloss:0.50301
[10]	train-logloss:0.46989	eval-logloss:0.49287
[11]	train-logloss:0.45974	eval-logloss:0.48561
[12]	train-logloss:0.44967	eval-logloss:0.47577
[13]	train-logloss:0.44073	eval-logloss:0.46818
[14]	train-logloss:0.43264	eval-logloss:0.46098
[15]	train-logloss:0.42565	eval-logloss:0.45468
[16]	train-logloss:0.41900	eval-logloss:0.44939
[17]	train-logloss:0.41307	eval-logloss:0.44434
[18]	train-logloss:0.40845	eval-logloss:0.44150
[19]	train-logloss:0.40337	eval-logloss:0.43891
[20]	train-logloss:0.39911	eval-logloss:0.43656
[2

[I 2023-10-24 08:27:25,328] Trial 1 finished with value: 0.40877725639739515 and parameters: {'eta': 0.08117818483929862, 'max_depth': 5, 'min_child_weight': 4.432113391500656, 'colsample_bytree': 0.912120550273728, 'subsample': 0.9372819661753602}. Best is trial 1 with value: 0.40877725639739515.


...

[I 2023-10-24 08:27:37,265] Trial 37 finished with value: 0.4221132366850389 and parameters: {'eta': 0.05858148386307203, 'max_depth': 5, 'min_child_weight': 4.946534475221827, 'colsample_bytree': 0.7186447908729223, 'subsample': 0.6628215913212495}. Best is trial 14 with value: 0.40235291577501003.


[0]	train-logloss:0.66643	eval-logloss:0.66684
[1]	train-logloss:0.65087	eval-logloss:0.65623
[2]	train-logloss:0.62775	eval-logloss:0.63380
[3]	train-logloss:0.61449	eval-logloss:0.62470
[4]	train-logloss:0.59508	eval-logloss:0.60678
[5]	train-logloss:0.57801	eval-logloss:0.59061
[6]	train-logloss:0.56894	eval-logloss:0.58565
[7]	train-logloss:0.55251	eval-logloss:0.57174
[8]	train-logloss:0.53803	eval-logloss:0.55846
[9]	train-logloss:0.52459	eval-logloss:0.54850
[10]	train-logloss:0.51294	eval-logloss:0.53761
[11]	train-logloss:0.50214	eval-logloss:0.52878
[12]	train-logloss:0.49247	eval-logloss:0.51832
[13]	train-logloss:0.48106	eval-logloss:0.50815
[14]	train-logloss:0.47182	eval-logloss:0.49939
[15]	train-logloss:0.46432	eval-logloss:0.49397
[16]	train-logloss:0.45556	eval-logloss:0.48545
[17]	train-logloss:0.44857	eval-logloss:0.48041
[18]	train-logloss:0.44184	eval-logloss:0.47458
[19]	train-logloss:0.43641	eval-logloss:0.46903
[20]	train-logloss:0.42940	eval-logloss:0.46523
[2

[I 2023-10-24 08:27:37,615] Trial 38 finished with value: 0.41446535561920544 and parameters: {'eta': 0.058549329934156576, 'max_depth': 5, 'min_child_weight': 2.3168544117977636, 'colsample_bytree': 0.7208730882439159, 'subsample': 0.9248284527129568}. Best is trial 14 with value: 0.40235291577501003.


[0]	train-logloss:0.65860	eval-logloss:0.65678
[1]	train-logloss:0.64394	eval-logloss:0.64082
[2]	train-logloss:0.61743	eval-logloss:0.61274
[3]	train-logloss:0.60606	eval-logloss:0.60027
[4]	train-logloss:0.58576	eval-logloss:0.58049
[5]	train-logloss:0.56707	eval-logloss:0.56041
[6]	train-logloss:0.55916	eval-logloss:0.55432
[7]	train-logloss:0.54289	eval-logloss:0.53744
[8]	train-logloss:0.52931	eval-logloss:0.52418
[9]	train-logloss:0.51903	eval-logloss:0.51423
[10]	train-logloss:0.50741	eval-logloss:0.50127
[11]	train-logloss:0.49882	eval-logloss:0.49388
[12]	train-logloss:0.49035	eval-logloss:0.48465
[13]	train-logloss:0.48144	eval-logloss:0.47545
[14]	train-logloss:0.47354	eval-logloss:0.46964
[15]	train-logloss:0.46812	eval-logloss:0.46590
[16]	train-logloss:0.46197	eval-logloss:0.46046
[17]	train-logloss:0.45746	eval-logloss:0.45683
[18]	train-logloss:0.45347	eval-logloss:0.45545
[19]	train-logloss:0.44955	eval-logloss:0.45088
[20]	train-logloss:0.44530	eval-logloss:0.44821
[2

[I 2023-10-24 08:27:37,851] Trial 39 finished with value: 0.41965244337916374 and parameters: {'eta': 0.08522072009617665, 'max_depth': 4, 'min_child_weight': 7.858078028488696, 'colsample_bytree': 0.7750092042052554, 'subsample': 0.6176295198651403}. Best is trial 14 with value: 0.40235291577501003.


In [12]:
params = {
    'objective': 'binary:logistic'
}

for i, j in study.best_params.items():
    params[i] = j

params

{'objective': 'binary:logistic',
 'eta': 0.0974785526725371,
 'max_depth': 8,
 'min_child_weight': 0.23435919255988735,
 'colsample_bytree': 0.8179961334413343,
 'subsample': 0.8357808707925263}

## Model

In [13]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_valid, label=y_valid)
    
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
num_round = 500
    
model = xgb.train(
    params,
    dtrain,
    num_round,
    evals=watchlist,
    early_stopping_rounds=20
)

[08:27:46] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	train-logloss:0.64604	eval-logloss:0.65131
[1]	train-logloss:0.61343	eval-logloss:0.62955
[2]	train-logloss:0.57528	eval-logloss:0.59522
[3]	train-logloss:0.55057	eval-logloss:0.58236
[4]	train-logloss:0.51776	eval-logloss:0.55700
[5]	train-logloss:0.49178	eval-logloss:0.53479
[6]	train-logloss:0.47805	eval-logloss:0.52906
[7]	train-logloss:0.45837	eval-logloss:0.51315
[8]	train-logloss:0.43623	eval-logloss:0.49552
[9]	train-logloss:0.42126	eval-logloss:0.48537
[10]	train-logloss:0.40651	eval-logloss:0.47500
[11]	train-logloss:0.38947	eval-logloss:0.46424
[12]	train-logloss:0.37755	eval-logloss:0.45573
[13]	train-logloss:0.36423	eval-logloss:0.44694
[14]	train-logloss:0.35255	eval-logloss:0.43979
[15]	train-logloss:0.34394	eval-loglos

## Evaluate

In [14]:
y_pred_valid = model.predict(dvalid, ntree_limit=model.best_ntree_limit)
y_pred_valid = (y_pred_valid > 0.5).astype(int)

In [15]:
f1_score(y_valid, y_pred_valid, average='macro')

0.8091684434968016

## Submit

In [16]:
X_test = test[features]

In [17]:
dtest = xgb.DMatrix(X_test)
y_pred_test = model.predict(dtest, ntree_limit=model.best_ntree_limit)
y_pred_test = (y_pred_test > 0.5).astype(int)

In [18]:
submit = pd.DataFrame(y_pred_test, index=test['PassengerId'], columns=['Survived'])
submit.to_csv('submit.csv')